In [1]:
import pathlib
from typing import Dict, List, Union, Optional
from urllib.request import urlretrieve
import re
import requests
import os

from bs4 import BeautifulSoup
import contextily as cx
import geopandas as gpd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point, Polygon

%matplotlib inline

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 150
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
load_dotenv()
DATA_DIR = pathlib.Path("data").resolve()

In [4]:
def extract_data_from_url(
    file_path: pathlib.Path,
    url: str,
    data_format: str,
    force_repull: bool = False,
) -> pd.DataFrame:
    file_path = pathlib.Path(file_path).resolve()
    file_path.parent.mkdir(exist_ok=True, parents=True)
    if not file_path.is_file() or force_repull:
        urlretrieve(url, file_path)
    if data_format in ["csv", "zipped_csv"]:
        return pd.read_csv(file_path)
    elif data_format in ["json"]:
        return pd.read_json(file_path)
    elif data_format in ["shp", "geojson"]:
        return gpd.read_file(file_path)
    
def decode_lines(response_lines):
    decoded_lines = []
    error_lines = []
    for line in response_lines:
        try:
            decoded_lines.append(line.decode("utf-8"))
        except Exception:
            error_lines.append(line)
            print(line)
    return (decoded_lines, error_lines)
    
def get_edgar_cik_lookup_data(file_path: pathlib.Path = DATA_DIR.joinpath("edgar-cik-lookup-data.csv"), force_repull: bool = False) -> pd.DataFrame:
    if not file_path.is_file() or force_repull:
        resp = requests.get(
            url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt",
            headers={"User-Agent":os.environ["email"]}
        )
        if resp.status_code == 200:
            response_lines = [line for line in resp.iter_lines()]
            decoded_lines, error_lines = decode_lines(response_lines)
            company_cik_codes = pd.DataFrame(decoded_lines)
            assert all(company_cik_codes[0].str[-11:-8] == "000"), (
                "A company had either a cik id >= 10^7 or a malformed cik. Investigate."
            )
            company_cik_codes_df = pd.DataFrame(
                {"company": company_cik_codes[0].str[:-12],
                 "cik": company_cik_codes[0].str[-11:-1]}
            )
            company_cik_codes_df.to_csv(file_path, index=False)
        else:
            raise Exception(f"Failed to get a valid response: {rest.status_code}")
    return pd.read_csv(file_path, dtype=str)

In [5]:
company_tickers_df = extract_data_from_url(
    file_path = DATA_DIR.joinpath("company_tickers.json"),
    url = "https://www.sec.gov/files/company_tickers.json",
    data_format="json",
)
company_tickers_df = company_tickers_df.T

In [6]:
print(company_tickers_df.shape)
company_tickers_df.head(2)

(12115, 3)


,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP


In [7]:
company_cik_codes_df = get_edgar_cik_lookup_data(file_path=DATA_DIR.joinpath("edgar-cik-lookup-data.csv"))

In [8]:
print(company_cik_codes_df.shape)
company_cik_codes_df.head(2)

(881109, 2)


,company,cik
0,!J INC,0001438823
1,"#1 A LIFESAFER HOLDINGS, INC.",0001509607


In [9]:
base_url = "https://www.sec.gov/Archives/edgar/Feed/"

resp = requests.get(base_url, headers={"User-Agent": os.environ["email"]})

In [10]:
soup = BeautifulSoup(resp.content)
tables = soup.find_all("table")
assert len(tables) == 1, (
    f"More tables than expected found on the page at {base_url}. Investigate."
)
year_table = tables[0]
year_table_rows = year_table.find_all("tr")

In [14]:
year_table_rows

[<tr><th>Name</th><th>Size</th><th>Last Modified</th>
 </tr>,
 <tr><td><a href="1995/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1995</a></td><td> </td><td>05/02/2017 08:59:46 AM</td></tr>,
 <tr><td><a href="1996/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1996</a></td><td> </td><td>05/02/2017 09:02:18 AM</td></tr>,
 <tr><td><a href="1997/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1997</a></td><td> </td><td>05/02/2017 09:05:27 AM</td></tr>,
 <tr><td><a href="1998/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1998</a></td><td> </td><td>05/02/2017 09:09:46 AM</td></tr>,
 <tr><td><a href="1999/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1999</a></td><td> </td><td>05/02/2017 09:10:23 AM</td></tr>,
 <tr><td><a href="2000/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>2000</a></td><td> </td><td>03/28/2018 08:13:38 AM</td></tr>,
 <tr><td><a href="2001/"><img al

In [26]:
table_header_line = year_table_rows[0].find_all("th")
table_header = [el.text for el in table_header_line]
table_header

['Name', 'Size', 'Last Modified']

['Name', 'Size', 'Last Modified']

In [27]:
table_data_row = year_table_rows[1].find_all("td")
table_data_row

[<td><a href="1995/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1995</a></td>,
 <td> </td>,
 <td>05/02/2017 08:59:46 AM</td>]

In [ ]:
table_data_row[0]

In [13]:
dir(year_table_rows[0])

['DEFAULT_INTERESTING_STRING_TYPES',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_namespaces',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'default',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings'

In [14]:

year_table

<table summary="heding"><tr><th>Name</th><th>Size</th><th>Last Modified</th>
</tr><tr><td><a href="1995/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1995</a></td><td> </td><td>05/02/2017 08:59:46 AM</td></tr>
<tr><td><a href="1996/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1996</a></td><td> </td><td>05/02/2017 09:02:18 AM</td></tr>
<tr><td><a href="1997/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1997</a></td><td> </td><td>05/02/2017 09:05:27 AM</td></tr>
<tr><td><a href="1998/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1998</a></td><td> </td><td>05/02/2017 09:09:46 AM</td></tr>
<tr><td><a href="1999/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1999</a></td><td> </td><td>05/02/2017 09:10:23 AM</td></tr>
<tr><td><a href="2000/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>2000</a></td><td> </td><td>03/28/2018 08:13:38 AM</td></tr>
<tr><td><a href="2001/">

In [21]:
year_table_rows

[<tr><th>Name</th><th>Size</th><th>Last Modified</th>
 </tr>,
 <tr><td><a href="1995/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1995</a></td><td> </td><td>05/02/2017 08:59:46 AM</td></tr>,
 <tr><td><a href="1996/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1996</a></td><td> </td><td>05/02/2017 09:02:18 AM</td></tr>,
 <tr><td><a href="1997/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1997</a></td><td> </td><td>05/02/2017 09:05:27 AM</td></tr>,
 <tr><td><a href="1998/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1998</a></td><td> </td><td>05/02/2017 09:09:46 AM</td></tr>,
 <tr><td><a href="1999/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1999</a></td><td> </td><td>05/02/2017 09:10:23 AM</td></tr>,
 <tr><td><a href="2000/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>2000</a></td><td> </td><td>03/28/2018 08:13:38 AM</td></tr>,
 <tr><td><a href="2001/"><img al

In [16]:
year_table.text

'NameSizeLast Modified\n1995\xa005/02/2017 08:59:46 AM\n1996\xa005/02/2017 09:02:18 AM\n1997\xa005/02/2017 09:05:27 AM\n1998\xa005/02/2017 09:09:46 AM\n1999\xa005/02/2017 09:10:23 AM\n2000\xa003/28/2018 08:13:38 AM\n2001\xa003/28/2018 08:15:42 AM\n2002\xa003/28/2018 08:15:50 AM\n2003\xa003/28/2018 08:15:57 AM\n2004\xa003/28/2018 08:16:04 AM\n2005\xa003/28/2018 08:16:10 AM\n2006\xa003/28/2018 08:16:16 AM\n2007\xa003/28/2018 08:16:23 AM\n2008\xa003/28/2018 08:16:30 AM\n2009\xa003/28/2018 08:16:39 AM\n2010\xa005/22/2018 08:54:02 AM\n2011\xa005/22/2018 08:54:53 AM\n2012\xa005/22/2018 08:55:05 AM\n2013\xa005/22/2018 08:55:13 AM\n2014\xa005/22/2018 08:55:23 AM\n2015\xa005/22/2018 08:56:13 AM\n2016\xa005/22/2018 08:56:48 AM\n2017\xa005/22/2018 08:56:57 AM\n2018\xa002/27/2019 08:40:24 AM\n2019\xa007/13/2020 03:03:24 PM\n2020\xa011/11/2020 09:26:39 AM\n2021\xa001/28/2022 09:45:39 PM\n2022\xa004/01/2022 12:20:15 AM\n'

In [17]:
# installed lxml and html5lib
table_df = pd.read_html(year_table)

TypeError: 'NoneType' object is not callable

In [80]:
[el for el in year_table.children]

[<tr><th>Name</th><th>Size</th><th>Last Modified</th>
 <tr><td><a href="1995/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1995</a></td><td> </td><td>05/02/2017 08:59:46 AM</td></tr>
 <tr><td><a href="1996/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1996</a></td><td> </td><td>05/02/2017 09:02:18 AM</td></tr>
 <tr><td><a href="1997/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1997</a></td><td> </td><td>05/02/2017 09:05:27 AM</td></tr>
 <tr><td><a href="1998/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1998</a></td><td> </td><td>05/02/2017 09:09:46 AM</td></tr>
 <tr><td><a href="1999/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1999</a></td><td> </td><td>05/02/2017 09:10:23 AM</td></tr>
 <tr><td><a href="2000/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>2000</a></td><td> </td><td>03/28/2018 08:13:38 AM</td></tr>
 <tr><td><a href="2001/"><img alt="folder icon

<tr><th>Name</th><th>Size</th><th>Last Modified</th>
<tr><td><a href="1995/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1995</a></td><td> </td><td>05/02/2017 08:59:46 AM</td></tr>
<tr><td><a href="1996/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1996</a></td><td> </td><td>05/02/2017 09:02:18 AM</td></tr>
<tr><td><a href="1997/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1997</a></td><td> </td><td>05/02/2017 09:05:27 AM</td></tr>
<tr><td><a href="1998/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1998</a></td><td> </td><td>05/02/2017 09:09:46 AM</td></tr>
<tr><td><a href="1999/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>1999</a></td><td> </td><td>05/02/2017 09:10:23 AM</td></tr>
<tr><td><a href="2000/"><img alt="folder icon" class="img_icon" src="/icons/folder.gif"/>2000</a></td><td> </td><td>03/28/2018 08:13:38 AM</td></tr>
<tr><td><a href="2001/"><img alt="folder icon" class=

In [84]:
len(list(year_table.children))
# [0]

1

In [78]:
??year_table.children

Type:        property
String form: <property object at 0x7f87c8cfcf40>
Source:     
# year_table.children.fget
@property
def children(self):
    """Iterate over all direct children of this PageElement.

    :yield: A sequence of PageElements.
    """
    # return iter() to make the purpose of the method clear
    return iter(self.contents)  # XXX This seems to be untested.


In [73]:
dir(year_table)

['DEFAULT_INTERESTING_STRING_TYPES',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_namespaces',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'default',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings'

In [31]:
resp = requests.get(
    url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt",
    headers={"User-Agent":os.environ["email"]}
)
if resp.status_code == 200:
    response_lines = [line for line in resp.iter_lines()]
    decoded_lines, error_lines = decode_lines(response_lines)
    company_cik_codes = pd.DataFrame(decoded_lines)
    assert all(company_cik_codes[0].str[-11:-8] == "000"), (
        "A company had either a cik id >= 10^7 or a malformed cik. Investigate."
    )
    company_cik_codes_df = pd.DataFrame({
        "company": company_cik_codes[0].str[-11:]
        "cik": company_cik_codes[0].str[-11:],
    })
    # company_cik_codes_df = company_cik_codes[0].str.split(":", n=1, expand=True)
    # company_cik_codes_df = company_cik_codes_df.rename(columns={0:"company", 1:"cik"})
    # company_cik_codes_df["cik"] = company_cik_codes_df["cik"].str.replace(":", "")

b'EXPLORA INVESTIMENTOS GEST\xc3O DE RECURSOS LTDA:0001538467:'
b'PRAGMA GEST\xc3O DE PATRIM\xd4NIO LTDA:0001555512:'


In [41]:
company_cik_codes_df = pd.DataFrame({
    "company": company_cik_codes[0].str[:-12],
    "cik": company_cik_codes[0].str[-11:-1],
})

In [42]:
company_cik_codes_df

,company,cik
0,!J INC,0001438823
1,"#1 A LIFESAFER HOLDINGS, INC.",0001509607
2,#1 ARIZONA DISCOUNT PROPERTIES LLC,0001457512
3,#1 PAINTBALL CORP,0001433777
4,$ LLC,0001427189
...,...,...
881104,ZZIF 2008 INVESTMENT LLC,0001448632
881105,"ZZLL INFORMATION TECHNOLOGY, INC",0001365357
881106,"ZZX, LLC",0001691924
881107,ZZYZX ZZAZX ZZOZX INC,0001184274


In [32]:
print(company_cik_codes.head(2))

                                           0
0                         !J INC:0001438823:
1  #1 A LIFESAFER HOLDINGS, INC.:0001509607:


In [33]:
company_cik_codes[0].str[-1].value_counts(dropna=False)

:    881109
Name: 0, dtype: int64

In [37]:
company_cik_codes[0].str[-11:-8].value_counts(dropna=False)

000    881109
Name: 0, dtype: int64

In [46]:
assert all(company_cik_codes[0].str[-11:-8] == "000")

In [34]:
import re

In [35]:
cik_num_pattern = re.compile("(\d{10}:$)")

In [ ]:
re.findall()

In [76]:
url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt"

# urlretrieve(url=url, filename=out_path)

In [1]:
def get_edgar_cik_lookup_data(file_path: pathlib.Path = DATA_DIR.joinpath("edgar-cik-lookup-data.csv")) -> pd.DataFrame:
    if not file_path.is_file():
        resp = requests.get(
            url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt",
            headers={"User-Agent":os.environ["email"]}
        )
        if resp.status_code == 200:
            response_lines = [line for line in resp.iter_lines()]
            decoded_lines, error_lines = decode_lines(response_lines)
            company_cik_codes = pd.DataFrame(decoded_lines)
            company_cik_codes_df = company_cik_codes[0].str.split(":", n=1, expand=True)
            company_cik_codes_df = company_cik_codes_df.rename(columns={0:"company", 1:"cik"})
            company_cik_codes_df["cik"] = company_cik_codes_df["cik"].str.replace(":", "")
            company_cik_codes_df.to_csv(file_path, index=False)
        else:
            raise Exception(f"Failed to get a valid response: {rest.status_code}")
    return pd.read_csv(file_path)

NameError: name 'DATA_DIR' is not defined

In [80]:
url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt"
resp = requests.get(
    url,
    headers={
        "User-Agent":os.environ["email"]
    }
)

In [81]:
resp.status_code

200

In [82]:
response_lines = [line for line in resp.iter_lines()]
decoded_lines, error_lines = decode_lines(response_lines)
company_cik_codes = pd.DataFrame(decoded_lines)
company_cik_codes_df = company_cik_codes[0].str.split(":", n=1, expand=True)
company_cik_codes_df = company_cik_codes_df.rename(columns={0:"company", 1:"cik"})
company_cik_codes_df["cik"] = company_cik_codes_df["cik"].str.replace(":", "")

b'EXPLORA INVESTIMENTOS GEST\xc3O DE RECURSOS LTDA:0001538467:'
b'PRAGMA GEST\xc3O DE PATRIM\xd4NIO LTDA:0001555512:'


In [83]:
print(company_cik_codes.shape)
company_cik_codes.head(2)

(881109, 1)


,0
0,!J INC:0001438823:
1,"#1 A LIFESAFER HOLDINGS, INC.:0001509607:"


In [90]:
company_cik_codes_df.to_csv(DATA_DIR.joinpath("edgar-cik-lookup-data.csv"), index=False)

In [89]:
company_cik_codes_df.head(2)

,company,cik
0,!J INC,0001438823
1,"#1 A LIFESAFER HOLDINGS, INC.",0001509607


In [36]:
response_lines = [line for line in resp.iter_lines()]

In [40]:
# response_lines[20:40]

In [ ]:
company_cik_codes

In [59]:
# [line.split(":", 1) for line in decoded_lines]

In [60]:
# decoded_lines

In [46]:
print(error_lines[0])

b'EXPLORA INVESTIMENTOS GEST\xc3O DE RECURSOS LTDA:0001538467:'
